In [1]:
#Load libraries
import os
import csv  
from shutil import copyfile
import pandas as pd
import torchvision
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image
import rasterio

In [2]:
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
#print(base_dir)
city= "ams"
#path to ancillary data folder
ancillary_data_folder_path = base_dir + "/data_prep/{}_Projectdata/AncillaryData".format(city)
ancillary_POPdata_folder_path = base_dir + "/data_prep/{}_Projectdata/PopData".format(city)
data_folder =  base_dir + "/model/pytorch_model/data"

In [3]:
## ## ## ## ## ----- CREATE NEW FOLDER  ----- ## ## ## ## ##
def createFolder(path):
    if not os.path.exists(path):
        print("------------------------------ Creating Folder : {} ------------------------------".format(path))
        os.makedirs(path)
    else: 
        print("------------------------------ Folder already exists------------------------------")

In [4]:
img_dir = base_dir + "/model/pytorch_model/img_dir"
createFolder(img_dir) 

------------------------------ Folder already exists------------------------------


In [ ]:
classes = []

for folder in os.listdir(data_folder):
    classes.append(folder)
    annotation = len(classes) - 1
    #print(len(classes))
    for file in os.listdir(data_folder+ "/" + folder):
        if file.endswith(".tif"):
            year = file.split(".")[0]
            #print(file)
            src = data_folder+ "/" + folder + "/" + file
            dst = img_dir + "/{0}_{1}.tif".format(folder,year) 
            #copyfile(src, dst)
            #row_data = []
            
            with open('labels.csv', mode='a', encoding='UTF8') as f:
                f.write("{0}_{1}.tif, {2}".format(folder,year, annotation))
                f.write("\n")  # Next line.


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import IMG_EXTENSIONS

#len(ImageFolder(img_dir))  # prints 1

IMG_EXTENSIONS.append('tif')

len(ImageFolder(img_dir))  # prints 2

In [29]:
annotations_file = 'labels.csv'

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        src = rasterio.open(img_path)
        array = src.read(1)
        image = ToTensor()(array)
        label = self.img_labels.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
        
    

In [36]:
train_dataset = CustomImageDataset(annotations_file, img_dir)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CustomImageDataset(annotations_file, img_dir)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [1]:
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

NameError: name 'train_loader' is not defined